In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
%matplotlib inline
pd.options.display.float_format = '{:,.2f}'.format
plt.style.use('seaborn-white')
# colorblind safe
plt.style.use('seaborn-colorblind')
plt.style.use('tableau-colorblind10')

# width and precision for f strings
width = 10
precision = 4

# default sizes for plots
# https://matplotlib.org/3.3.0/tutorials/introductory/customizing.html#customizing-with-matplotlibrc-files
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['font.size'] = 16
plt.rcParams['legend.fontsize'] = 'large'
plt.rcParams['figure.titlesize'] = 'medium'
plt.rcParams['lines.linewidth'] = 2

# other settings
pd.options.display.float_format = '{:,.4f}'.format
pd.set_option("display.precision", 3)
np.set_printoptions(precision=3, suppress=True)
%load_ext autoreload
%autoreload 2
pd.set_option('display.max_columns', None)
%config IPCompleter.greedy=True

## notes
* book events some are fraudulent
* build model and put into production
* deploy to AWS
* web service that can pull online
* give probability of fraud?
* or why we think fraud for each
* dashboard shows all fraud
* and then user can categorize if fraud
* and save it in database
* anomaly detection using supervised classification
* make plots


In [3]:
data = pd.read_json('../data/data.zip')

In [21]:
data.to_csv('../data/data_as_csv', index=False)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14337 entries, 0 to 14336
Data columns (total 44 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   acct_type           14337 non-null  object 
 1   approx_payout_date  14337 non-null  int64  
 2   body_length         14337 non-null  int64  
 3   channels            14337 non-null  int64  
 4   country             14256 non-null  object 
 5   currency            14337 non-null  object 
 6   delivery_method     14321 non-null  float64
 7   description         14337 non-null  object 
 8   email_domain        14337 non-null  object 
 9   event_created       14337 non-null  int64  
 10  event_end           14337 non-null  int64  
 11  event_published     14238 non-null  float64
 12  event_start         14337 non-null  int64  
 13  fb_published        14337 non-null  int64  
 14  gts                 14337 non-null  float64
 15  has_analytics       14337 non-null  int64  
 16  has_

In [5]:
data.head()

,acct_type,approx_payout_date,body_length,channels,country,currency,delivery_method,description,email_domain,event_created,event_end,event_published,event_start,fb_published,gts,has_analytics,has_header,has_logo,listed,name,name_length,num_order,num_payouts,object_id,org_desc,org_facebook,org_name,org_twitter,payee_name,payout_type,previous_payouts,sale_duration,sale_duration2,show_map,ticket_types,user_age,user_created,user_type,venue_address,venue_country,venue_latitude,venue_longitude,venue_name,venue_state
0,fraudster_event,1266062400,3852,5,US,USD,0.0000,"<p><a href=""http://s432.photobucket.com/albums...",gmail.com,1262739706,1265630400,"1,263,109,838.0000",1265594400,0,0.0000,0,1.0000,0,y,"99 HOUR ""NO SLEEP"" SUPER BOWL CELEBRITY WEEKEN...",60,0,0,527017,,0.0000,Party Starz Ent & Diverse Int'l Group,0.0000,,,[],29.0000,33,1,"[{'event_id': 527017, 'cost': 25.0, 'availabil...",36,1259613950,1,717 Washington Avenue,US,25.7775,-80.1334,INK Nightclub - South Beach,FL
1,premium,1296720000,3499,0,US,USD,1.0000,"<p>Join us for a quick, one-night, community-b...",ruf.org,1293832670,1296288000,"1,293,833,295.0000",1296255600,0,868.0200,0,0.0000,1,n,Winthrop RUF Winter Getaway,27,23,1,786878,"<p>Since 1987, RUF has ministered to students ...",0.0000,RUF at Winthrop University,12.0000,RUF,CHECK,"[{'name': 'RUF', 'created': '2010-10-01 01:10:...",28.0000,28,0,"[{'event_id': 786878, 'cost': 35.0, 'availabil...",149,1280942776,3,,US,32.7766,-79.9309,"The Charleston, SC area",SC
2,premium,1296172800,2601,8,US,USD,1.0000,"<h3><span class=""subcategory""><strong>Teacher ...",pvsd.k12.ca.us,1291090956,1295740800,"1,291,091,908.0000",1295713800,0,"3,500.0000",0,nan,0,y,DRDP (2010) Teacher Training,28,19,4,787337,"<p><a href=""http://www.desiredresults.com"">www...",0.0000,University Preparation School in collaboration...,0.0000,University Preparation School,CHECK,"[{'name': 'Danielle Severn', 'created': '2010-...",54.0000,54,0,"[{'event_id': 787337, 'cost': 93.51, 'availabi...",214,1272559388,3,10100 Pioneer Blvd Suite 100,US,33.9442,-118.0804,Los Angeles County Office of Education,CA
3,premium,1388966400,12347,6,IE,EUR,1.0000,"<p style=""margin-bottom: 1.3em; padding-bottom...",irishtabletennis.com,1360681570,1388534400,"1,360,683,446.0000",1360702800,0,"1,167.3500",0,0.0000,1,y,ITTA Affiliation 2013,21,39,16,885645,,0.0000,,0.0000,,ACH,"[{'name': '', 'created': '2010-11-09 01:10:15'...",0.0000,0,0,"[{'event_id': 885645, 'cost': 25.0, 'availabil...",889,1283870102,3,,None,nan,nan,None,None
4,premium,1297900800,2417,11,US,USD,0.0000,<p>Writers and filmmakers need to understand t...,artsandbusinesscouncil.org,1291994666,1297468800,"1,291,994,966.0000",1297440000,1,"2,313.1500",0,0.0000,0,y,Self Defense for Writers and Filmmakers: For A...,66,30,0,1114349,"<p><font face=""Arial""><font size=""2""><strong>T...",0.0000,Arts & Business Council of Greater Boston,0.0000,Arts and Business Council or Greater Boston,CHECK,[{'name': 'Arts and Business Council or Greate...,63.0000,63,0,"[{'event_id': 1114349, 'cost': 150.0, 'availab...",35,1288984065,3,One Marina Park Drive,US,42.3538,-71.0443,Fish & Richardson,MA


In [6]:
data['acct_type'].unique()

array(['fraudster_event', 'premium', 'spammer_warn', 'fraudster',
       'spammer_limited', 'spammer_noinvite', 'locked', 'tos_lock',
       'tos_warn', 'fraudster_att', 'spammer_web', 'spammer'],
      dtype=object)

In [7]:
data.groupby(by='acct_type').agg('count')

,approx_payout_date,body_length,channels,country,currency,delivery_method,description,email_domain,event_created,event_end,event_published,event_start,fb_published,gts,has_analytics,has_header,has_logo,listed,name,name_length,num_order,num_payouts,object_id,org_desc,org_facebook,org_name,org_twitter,payee_name,payout_type,previous_payouts,sale_duration,sale_duration2,show_map,ticket_types,user_age,user_created,user_type,venue_address,venue_country,venue_latitude,venue_longitude,venue_name,venue_state
acct_type,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
fraudster,437,437,437,437,437,425,437,437,437,437,423,437,437,437,437,272,437,437,437,437,437,437,437,437,413,437,413,437,437,437,418,437,437,437,437,437,437,437,330,330,330,330,330
fraudster_att,5,5,5,5,5,5,5,5,5,5,4,5,5,5,5,3,5,5,5,5,5,5,5,5,4,5,4,5,5,5,4,5,5,5,5,5,5,5,4,4,4,4,4
fraudster_event,851,851,851,851,851,851,851,851,851,851,772,851,851,851,851,244,851,851,851,851,851,851,851,851,821,851,821,851,851,851,753,851,851,851,851,851,851,851,589,589,589,589,589
locked,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,41,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,51,51,51,51,51
premium,12373,12373,12373,12297,12373,12369,12373,12373,12373,12373,12368,12373,12373,12373,12373,7950,12373,12373,12373,12373,12373,12373,12373,12373,12369,12373,12369,12373,12373,12373,12337,12373,12373,12373,12373,12373,12373,12373,11700,11700,11700,11700,11700
spammer,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
spammer_limited,218,218,218,214,218,218,218,218,218,218,218,218,218,218,218,144,218,218,218,218,218,218,218,218,218,218,218,218,218,218,217,218,218,218,218,218,218,218,204,204,204,204,204
spammer_noinvite,84,84,84,84,84,84,84,84,84,84,84,84,84,84,84,59,84,84,84,84,84,84,84,84,84,84,84,84,84,84,84,84,84,84,84,84,84,84,82,82,82,82,82
spammer_warn,144,144,144,143,144,144,144,144,144,144,144,144,144,144,144,101,144,144,144,144,144,144,144,144,144,144,144,144,144,144,144,144,144,144,144,144,144,144,140,140,140,140,140


In [ ]:
# check dates
pd.to_datetime(data['event_created'], origin='julian')

In [18]:
# not premium
data['acct_type'][~data['acct_type'].isin(['premium'])].agg('count')
# data.groupby(by='acct_type').agg('count')

1964

In [13]:
type(data['acct_type'][1])

str

In [19]:
data['acct_type'][~data['acct_type'].isin(['premium'])].agg('count') / len(data)

0.13698821231777916

In [20]:
 len(data)

14337